In [6]:
import argparse
import random
import tqdm
import numpy as np
import torch

import config
import dataset
from informer_model import InformerModel
from model import Model
from moment_model import MomentModel
from propose import ProposedModel
from evaluation import evaluate_mse, evaluate_nll

In [ ]:
def set_seed(seed: int) -> None:
    # random
    random.seed(seed)

    # numpy
    np.random.seed(seed)

    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.mps.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def load_moment_model(args: argparse.Namespace) -> Model:
    return MomentModel(param="AutonLab/MOMENT-1-large", pred_len=args.pred_len)


def load_informer_model(args: argparse.Namespace) -> Model:
    return InformerModel(args, checkpoint_path="checkpoints/informer.pth")
def load_proposed_model(moment_model: Model, informer_model: Model,input_size: int,train_dataset: torch.utils.data.Dataset, args: argparse.Namespace) -> Model:
    model = ProposedModel(moment_model=moment_model, informer_model=informer_model,input_size=input_size)
    model.train(train_dataset=train_dataset,args=args)
    torch.save(model, "checkpoints/proposed_model.pkt")
    return model

In [29]:
set_seed(0)

args = config.ARGS
train_dataset, test_dataset = dataset.load_dataset(args=args)
input_size = args.seq_len
moment_model = load_moment_model(args=args)
informer_model = load_informer_model(args=args)

In [30]:
proposed_model = load_proposed_model(moment_model, informer_model,input_size,test_dataset,args)

100%|██████████| 89/89 [03:32<00:00,  2.38s/it]


Epoch [1/1], Loss: 0.0611


In [31]:
# 予測結果の評価
results = {}

for method, model in {
    "informer": informer_model,
    "moment": moment_model,
    "proposed": proposed_model,
}.items():
    print(f"testing: {method}")
    test_dataloader = dataset.to_dataloader(test_dataset, args, "test")
    
    mse = 0
    nll = 0
    batch_limit = 100 # 評価するバッチ数の上限、ちゃんと評価するときは十分大きい値に設定する

    y_pred = []
    y_true = []
    for batch in tqdm.tqdm(test_dataloader):
        y_pred.append(model.predict_distr(batch).detach().tolist())
        y_true.append(batch[1][:, -1].squeeze().detach().tolist())

    y_pred, y_true = np.array(y_pred).reshape(-1, 2), np.array(y_true).flatten()
    results[method] = {
        "mse": evaluate_mse(y_pred, y_true),
        "nll": evaluate_nll(y_pred, y_true),
    }
    print(results[method])

    np.save(f"checkpoints/y_pred_{method}.npy", y_pred)
results

testing: informer


100%|██████████| 89/89 [02:40<00:00,  1.80s/it]


{'mse': 0.09626331544299449, 'nll': 16558.14542084187}
testing: moment


100%|██████████| 89/89 [38:22<00:00, 25.87s/it]


{'mse': 0.0734385307363973, 'nll': 1390.2025889021575}
testing: proposed


100%|██████████| 89/89 [41:07<00:00, 27.72s/it]

{'mse': 0.059914489442363175, 'nll': 995.2847888361615}


{'informer': {'mse': 0.09626331544299449, 'nll': 16558.14542084187},
 'moment': {'mse': 0.0734385307363973, 'nll': 1390.2025889021575},
 'proposed': {'mse': 0.059914489442363175, 'nll': 995.2847888361615}}

m=10の場合

'informer': {'mse': 0.09626331544299449, 'nll': 16558.14542084187}

'moment': {'mse': 0.0734385307363973, 'nll': 1390.2025889021575}

'proposed': {'mse': 0.059914489442363175, 'nll': 995.2847888361615}

In [ ]:
import pandas as pd

pd.DataFrame(results).to_csv("results.csv", index=False)

: 

test

In [20]:
import re

log_file_path = "Informer2020\e_layers1_d_layers1_dropout0.1_learning_rate0.001_embedlearned.log"  # ログファイルのパスを指定
file_name = "e_layers1_d_layers1_dropout0.1_learning_rate0.001_embedlearned.log"

mse_values = []
file_names = []
with open(log_file_path, 'r', encoding='utf-16') as file:
    for line in file:
        if "mse:" in line:
            try:
                # "mse:"の後の数値を抽出
                mse_value = float(line.split("mse:")[1].split(",")[0].strip())
                mse_values.append(mse_value)
                file_names.append(file_name)
            except ValueError:
                continue  # 数値変換できない行はスキップ
print(mse_values)
print(file_names)

[0.11141613870859146, 0.08897791057825089]
['e_layers1_d_layers1_dropout0.1_learning_rate0.001_embedlearned.log', 'e_layers1_d_layers1_dropout0.1_learning_rate0.001_embedlearned.log']
